In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from configs import TIMEZONE, LOG_FILE_NAME, set_logger
from datetime import datetime, timedelta
import pytz
import pandas as pd
import numpy as np
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import GetAssetsRequest
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest, StockBarsRequest, CryptoLatestQuoteRequest
from alpaca.data.requests import CryptoBarsRequest
from alpaca.trading.models import Order
from alpaca.data.timeframe import TimeFrame
from my_secrets import ALPACA_API_BASE_URL, PAPER_API_ID, PAPER_SECRET_KEY
import logging
import plotly.express as px
import plotly.graph_objects as go
set_logger()

In [15]:
from Trade_Class import Stock_Trader, Crypto_Trader
from Market_Monitor import Market_Monitor
from ALGO_crossover import bars_df_filter_dates, add_sma_columns, add_sma_crossovers

In [4]:
stock_trader = Stock_Trader(PAPER_API_ID, PAPER_SECRET_KEY, paper=True)
monitor = Market_Monitor(stock_trader.trading_client, TIMEZONE)

In [5]:
start = datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
end = datetime(year=2023, month=2, day=1, hour=0, minute=0, second=0)
bars_df = stock_trader.get_bars('AAPL', start=start, end=end, time_resolution='day')
bars_df.reset_index(inplace=True)
bars_df.sort_values(by=['timestamp'], ascending=True, inplace=True)
display(bars_df.head())

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2023-01-03 05:00:00+00:00,130.280,130.9000,124.17,125.07,112084324.0,1020782.0,125.725146
1,AAPL,2023-01-04 05:00:00+00:00,126.890,128.6557,125.08,126.36,89072202.0,769281.0,126.646333
2,AAPL,2023-01-05 05:00:00+00:00,127.130,127.7400,124.76,125.02,81246605.0,663745.0,126.088261
3,AAPL,2023-01-06 05:00:00+00:00,126.010,130.2900,124.89,129.62,87758343.0,711520.0,128.198067
4,AAPL,2023-01-09 05:00:00+00:00,130.465,133.4100,129.89,130.15,70832518.0,647372.0,131.628178


In [6]:
#could do 15 days as the short and 50 days as the long
#could do 15 mins as the short and 50 mins as the long

start = datetime(year=2023, month=1, day=3, hour=0, minute=0, second=0)
end = datetime(year=2023, month=1, day=5, hour=0, minute=0, second=0)
minutes_df = stock_trader.get_bars('AAPL', start=start, end=end, time_resolution='minute')
minutes_df.reset_index(inplace=True)
minutes_df.sort_values(by=['timestamp'], ascending=True, inplace=True)
display(minutes_df.head())

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2023-01-03 09:00:00+00:00,130.28,131.00,130.28,131.00,8174.0,208.0,130.854120
1,AAPL,2023-01-03 09:01:00+00:00,130.87,131.17,130.87,131.10,8820.0,157.0,130.955243
2,AAPL,2023-01-03 09:02:00+00:00,131.18,131.24,131.17,131.17,2112.0,53.0,131.208769
3,AAPL,2023-01-03 09:03:00+00:00,131.19,131.29,131.19,131.28,3888.0,90.0,131.220790
4,AAPL,2023-01-03 09:04:00+00:00,131.28,131.46,131.28,131.46,5984.0,88.0,131.327899


In [ ]:
class Crossover():
    def __init__(self, trading_client) -> None:
        self.trading_client = trading_client

        

In [ ]:
https://www.youtube.com/watch?v=uCfyrpu9nMk

In [9]:
minutes_df = add_sma_columns(minutes_df, fsma_period=10, ssma_period=100)
minutes_df = bars_df_filter_dates(minutes_df, monitor)
minutes_df = add_sma_crossovers(minutes_df, fsma_col='fast_sma', ssma_col='slow_sma')

In [13]:
minutes_df[minutes_df['bullish_crossover']].head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,slow_sma,fast_sma,prev_fsma,bullish_crossover,bearish_crossover
177,AAPL,2023-01-03 13:03:00+00:00,130.8100,130.8200,130.7500,130.7500,3719.0,62.0,130.787446,130.552800,130.56200,130.54900,True,False
197,AAPL,2023-01-03 13:23:00+00:00,130.7500,130.7900,130.7000,130.7000,5960.0,103.0,130.718970,130.481696,130.51097,130.47397,True,False
210,AAPL,2023-01-03 13:36:00+00:00,130.5101,130.6594,130.5101,130.6594,4601.0,70.0,130.594614,130.500882,130.50486,130.49392,True,False
211,AAPL,2023-01-03 13:37:00+00:00,130.5800,130.7000,130.5300,130.7000,7303.0,117.0,130.629860,130.507082,130.52586,130.50486,True,False
508,AAPL,2023-01-03 18:34:00+00:00,124.6309,124.7000,124.5400,124.5701,97539.0,1173.0,124.617185,124.616325,124.62403,124.59002,True,False


In [14]:
fig = px.line(minutes_df, x='timestamp', y=['close', 'slow_sma', 'fast_sma'])

for index, row in minutes_df[minutes_df['bullish_crossover']].iterrows():
    fig.add_vline(x=row.timestamp, line_dash='dash', line_color='green')

for index, row in minutes_df[minutes_df['bearish_crossover']].iterrows():
    fig.add_vline(x=row.timestamp, line_dash='dash', line_color='red')
fig.show()

## The stochastic oscillator

The stochastic oscillator is calculated by subtracting the low for the period (usually 14 days) from the current closing price, dividing by the total range for the period, and multiplying by 100.

When the stochastic crossover exceeds the 80 band, the financial instrument is determined to have been overbought. When the stochastic crossover drops below the 20 band, the underlying financial instrument is determined to have been oversold. This causes a sell signal to form. A buy signal is triggered when the crossover goes back up through the 20 band.


https://www.investopedia.com/terms/s/stochasticoscillator.asp#:~:text=The%20stochastic%20oscillator%20is%20calculated,period%2C%20and%20multiplying%20by%20100.

In [ ]:
def stochastic_oscillator(bars_df):
    period_high = bars_df['high'].max()
    period_low = bars_df['low'].min()
    current_close = bars_df['close'].iloc[-1]
    lane_momentum = 100*(current_close-period_low)/(period_high-period_low)
    return lane_momentum

In [ ]:
stochastic_oscillator(bars_df)

## The Relative Strength Index (RSI)

Meanwhile, the RSI tracks overbought and oversold levels by measuring the velocity of price movements. In other words, the RSI was designed to measure the speed of price movements, while the stochastic oscillator formula works best in consistent trading ranges.

https://www.investopedia.com/terms/r/rsi.asp

In [ ]:
def RSI(bars_df):
    

In [ ]:
difference = bars_df['close'] - bars_df['open']
difference_plus = difference[difference>0]
difference_minus = difference[difference<0]

average_gain = difference_plus.mean()
average_loss = difference_minus.abs().mean()

step_one = 100 - (100 / (1 + (average_gain/average_loss)))

In [ ]:
step_one

In [ ]:
bars_df